In [89]:
import pqmodel

In [112]:
import tensorflow as tf

from tensorflow import keras

model = tf.keras.Sequential()
# Adds a densely-connected layer with 64 units to the model:
model.add(layers.Dense(64, activation='relu'))
# Add another:
model.add(layers.Dense(64, activation='relu'))
# Add a softmax layer with 10 output units:
model.add(layers.Dense(29, activation='softmax'))

model.compile(optimizer=tf.keras.optimizers.Adam(0.01),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [113]:
trainDF = pqmodel.pqmodel_df(num_samples=30)
testDF = pqmodel.pqmodel_df(num_samples=10)

In [116]:
data = np.asarray(trainDF[[f's{i}' for i in range(3200)]])
labels = np.asarray(trainDF['class'])

val_data = np.asarray(testDF[[f's{i}' for i in range(3200)]])
val_labels = np.asarray(testDF['class'])

model.fit(data, labels, validation_data=(val_data, val_labels), epochs=10, batch_size=32)

Train on 870 samples, validate on 290 samples
Epoch 1/10
870/870 [==============================] - 0s 215us/sample - loss: 809140724.8184 - accuracy: 0.0000e+00 - val_loss: 835087404.1379 - val_accuracy: 0.0000e+00
Epoch 2/10
870/870 [==============================] - 0s 117us/sample - loss: 985056512.4414 - accuracy: 0.0000e+00 - val_loss: 1005677404.6897 - val_accuracy: 0.0000e+00
Epoch 3/10
870/870 [==============================] - 0s 111us/sample - loss: 1176635005.0575 - accuracy: 0.0000e+00 - val_loss: 1191969747.8621 - val_accuracy: 0.0000e+00
Epoch 4/10
870/870 [==============================] - 0s 114us/sample - loss: 1386196109.8299 - accuracy: 0.0000e+00 - val_loss: 1390377177.1586 - val_accuracy: 0.0000e+00
Epoch 5/10
870/870 [==============================] - 0s 103us/sample - loss: 1611084789.7011 - accuracy: 0.0000e+00 - val_loss: 1602066250.1517 - val_accuracy: 0.0000e+00
Epoch 6/10
870/870 [==============================] - 0s 96us/sample - loss: 1852541339.6598 - ac

In [117]:
model.evaluate(val_data, val_labels, batch_size=32)

290/290 [==============================] - 0s 67us/sample - loss: 2876898676.5241 - accuracy: 0.0000e+00


[2876898676.524138, 0.0]